<a href="https://colab.research.google.com/github/Fatihkuru57/Oyun-Projesi/blob/main/oyun1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import json
import random
import time
import sys
import os

# === Yavaş Yazma Fonksiyonu ===
def yavas_yaz(metin, hiz=0.02):
    for harf in metin:
        sys.stdout.write(harf)
        sys.stdout.flush()
        time.sleep(hiz)
    print()

# === Karakter Sınıfı ===
class Karakter:
    def __init__(self, isim, sinif="Savaşçı", veri=None):
        if veri:
            self.__dict__.update(veri)
        else:
            self.isim = isim
            self.sinif = sinif
            self.seviye = 1
            self.para = 50
            self.kat = 1
            self.envanter = []
            self.silah = None
            self.zirh = None
            self.buyuler = []
            self.xp = 0
            self.xp_sinir = 100
            if sinif == "Savaşçı":
                self.can = self.maksimum_can = 120
                self.guc = 12
                self.defans = 8
                self.hiz = 5
            elif sinif == "Okçu":
                self.can = self.maksimum_can = 100
                self.guc = 10
                self.defans = 5
                self.hiz = 8
            elif sinif == "Büyücü":
                self.can = self.maksimum_can = 80
                self.guc = 8
                self.defans = 4
                self.hiz = 6
                self.buyuler = ["Ateş Topu"]
            else:
                self.can = self.maksimum_can = 100
                self.guc = 10
                self.defans = 6
                self.hiz = 6

    def durum(self):
        print(f"\n--- {self.isim} Durumu ---")
        print(f"Seviye: {self.seviye} | Sınıf: {self.sinif} | XP: {self.xp}/{self.xp_sinir}")
        print(f"Can: {self.can}/{self.maksimum_can} | Güç: {self.guc} | Defans: {self.defans} | Hız: {self.hiz}")
        print(f"Para: {self.para} Altın")
        print(f"Silah: {self.silah if self.silah else 'Yok'}")
        print(f"Zırh: {self.zirh if self.zirh else 'Yok'}")
        print(f"Envanter: {', '.join(self.envanter) if self.envanter else 'Boş'}")
        print(f"Büyüler: {', '.join(self.buyuler) if self.buyuler else 'Yok'}")
        print(f"Zindan Katı: {self.kat}")
        print(f"Harita: {'🧍' + ' - ' * (self.kat - 1) + '⬛' * (10 - self.kat)}")

    def saldir(self, hedef):
        baz_hasar = self.guc
        if self.silah == "Kılıç":
            baz_hasar += 7
        elif self.silah == "Ok":
            baz_hasar += 5
        elif self.silah == "Asa":
            baz_hasar += 6

        kritik = random.random() < 0.1
        if kritik:
            baz_hasar *= 2
            yavas_yaz(f"*** Kritik vuruş! ***")

        net_hasar = max(0, baz_hasar - hedef.defans)
        hedef.can -= net_hasar

        yavas_yaz(f"{self.isim}, {hedef.isim}'a {net_hasar} hasar verdi!")

    def buyu_kullan(self, hedef):
        if not self.buyuler:
            yavas_yaz("Büyülerin yok!")
            return False
        yavas_yaz("Büyüler:")
        for i, b in enumerate(self.buyuler, 1):
            print(f"{i}. {b}")
        secim = input("Hangi büyüyü kullanmak istersin? (0 ile iptal) ")
        if secim == "0":
            return False
        if not secim.isdigit() or int(secim) < 1 or int(secim) > len(self.buyuler):
            yavas_yaz("Geçersiz seçim.")
            return False

        secilen_buyu = self.buyuler[int(secim) - 1]

        if secilen_buyu == "Ateş Topu":
            hasar = 25 + self.seviye * 3
            hedef.can -= hasar
            yavas_yaz(f"{self.isim}, {hedef.isim}'a Ateş Topu ile {hasar} büyü hasarı verdi!")
        elif secilen_buyu == "Şimşek":
            hasar = 30 + self.seviye * 2
            hedef.can -= hasar
            yavas_yaz(f"{self.isim}, {hedef.isim}'a Şimşek büyüsü ile {hasar} hasar verdi!")
        # Diğer büyüler eklenebilir

        return True

    def iyilestir(self):
        if "İksir" in self.envanter:
            yenilenme = 50 + self.seviye * 10
            self.can = min(self.maksimum_can, self.can + yenilenme)
            self.envanter.remove("İksir")
            yavas_yaz(f"{self.isim} iksir kullandı! Canı {yenilenme} puan yenilendi.")
        else:
            yavas_yaz("Envanterde iksir yok!")

    def envantere_ekle(self, urun):
        if urun not in self.envanter:
            self.envanter.append(urun)

    def seviye_atla(self):
        self.seviye += 1
        self.xp = 0
        self.xp_sinir = int(self.xp_sinir * 1.5)
        self.maksimum_can += 15
        self.can = self.maksimum_can
        self.guc += 3
        self.defans += 2
        self.hiz += 1
        yavas_yaz(f"\n🎉 Tebrikler! {self.isim} seviye atladı! Şimdi seviye {self.seviye}.\n")

# === Düşman Sınıfı ===
class Dusman:
    def __init__(self, seviye, boss=False):
        self.seviye = seviye
        self.boss = boss
        if boss:
            self.isim = random.choice(["Zindan Efendisi", "Ölüm Lordu", "Karanlık Büyücü"])
            self.can = 150 + seviye * 30
            self.guc = 20 + seviye * 5
            self.defans = 10 + seviye * 2
            self.hiz = 5 + seviye
        else:
            self.isim = random.choice(["Goblin", "Ork", "İskelet", "Zombi", "Kara Kurt", "Gölge Avcısı"])
            self.can = 40 + seviye * 15
            self.guc = 8 + seviye * 4
            self.defans = 4 + seviye * 2
            self.hiz = 4 + seviye

    def saldir(self, hedef):
        baz_hasar = self.guc
        kritik = random.random() < 0.1
        if kritik:
            baz_hasar *= 2
            yavas_yaz(f"*** {self.isim} kritik vurdu! ***")

        net_hasar = max(0, baz_hasar - hedef.defans)
        hedef.can -= net_hasar
        yavas_yaz(f"{self.isim}, {hedef.isim}'a {net_hasar} hasar verdi!")

# === Satıcı Sınıfı ===
class Satici:
    def __init__(self):
        self.urunler = {
            "İksir": 20,
            "Kılıç": 50,
            "Ok": 45,
            "Asa": 55,
            "Zırh": 60,
            "Büyü Kitabı": 100
        }

    def dükkan(self, oyuncu):
        yavas_yaz("\n[Satıcı] Al bakalım, bir şey lazım mı?")
        while True:
            for i, (urun, fiyat) in enumerate(self.urunler.items(), 1):
                print(f"{i}. {urun} - {fiyat} Altın")
            print("0. Çıkış")
            secim = input("Seçimin: ")
            if secim == "0":
                break
            elif secim.isdigit() and 1 <= int(secim) <= len(self.urunler):
                urun = list(self.urunler.keys())[int(secim) - 1]
                fiyat = self.urunler[urun]
                if oyuncu.para >= fiyat:
                    oyuncu.para -= fiyat
                    if urun in ["Kılıç", "Ok", "Asa"]:
                        oyuncu.silah = urun
                        yavas_yaz(f"{urun} satın alındı ve ekipmanına eklendi.")
                    elif urun == "Zırh":
                        if not oyuncu.zirh:
                            oyuncu.zirh = "Zırh"
                            oyuncu.defans += 5
                            yavas_yaz("Zırh satın alındı! Defansın arttı.")
                        else:
                            yavas_yaz("Zaten zırhın var.")
                    elif urun == "Büyü Kitabı":
                        if oyuncu.sinif == "Büyücü":
                            yeni_buyu = random.choice(["Şimşek", "Buz Mızrağı", "Ateş Topu"])
                            if yeni_buyu not in oyuncu.buyuler:
                                oyuncu.buyuler.append(yeni_buyu)
                                yavas_yaz(f"Yeni büyü öğrendin: {yeni_buyu}")
                            else:
                                yavas_yaz("Zaten bu büyüyü biliyorsun.")
                        else:
                            yavas_yaz("Sadece büyücüler büyü kitabı kullanabilir.")
                    else:
                        oyuncu.envanter.append(urun)
                        yavas_yaz(f"{urun} satın alındı ve envantere eklendi.")
                else:
                    yavas_yaz("Yeterli altın yok!")
            else:
                yavas_yaz("Geçersiz seçim.")

# === Kayıt/Yükleme ===
def kaydet(oyuncu):
    try:
        with open(f"{oyuncu.isim}.json", "w") as dosya:
            json.dump(oyuncu.__dict__, dosya)
        yavas_yaz("Oyun kaydedildi.")
    except Exception as e:
        yavas_yaz(f"Kaydetme hatası: {e}")

def yukle(isim):
    if not os.path.exists(f"{isim}.json"):
        yavas_yaz("Kayıt bulunamadı.")
        return None
    try:
        with open(f"{isim}.json", "r") as dosya:
            veri = json.load(dosya)
            return Karakter(isim=isim, veri=veri)
    except Exception as e:
        yavas_yaz(f"Yükleme hatası: {e}")
        return None

# === Rastgele Olay ===
def rastgele_olay(oyuncu):
    olay = random.choice(["dusman", "satici", "bos", "iyiles"])
    if olay == "dusman":
        dusman = Dusman(oyuncu.seviye, boss=(oyuncu.kat % 5 == 0))
        yavas_yaz(f"\nBir {dusman.isim} karşısına çıktı!")
        if combat(oyuncu, dusman):
            oyuncu.xp += 50 + dusman.seviye * 20
            oyuncu.para += 20 + dusman.seviye * 10
            yavas_yaz("Düşmanı yendin! XP ve altın kazandın.")
            if oyuncu.xp >= oyuncu.xp_sinir:
                oyuncu.seviye_atla()
            oyuncu.kat += 1
        else:
            oyuncu.can = 0  # Öldü
    elif olay == "satici":
        satici = Satici()
        satici.dükkan(oyuncu)
    elif olay == "iyiles":
        oyuncu.iyilestir()
    else:
        yavas_yaz("Herhangi bir şey olmadı, biraz dinlen.")

# === Savaş Fonksiyonu ===
def combat(oyuncu, dusman):
    yavas_yaz("\n--- Savaş Başladı! ---")
    while oyuncu.can > 0 and dusman.can > 0:
        yavas_yaz(f"\n{oyuncu.isim} Can: {oyuncu.can} | {dusman.isim} Can: {dusman.can}")
        print("1. Saldır")
        print("2. Büyü Kullan")
        print("3. İksir Kullan")
        print("4. Kaç")
        secim = input("Seçimin: ")

        if secim == "1":
            oyuncu.saldir(dusman)
        elif secim == "2":
            if oyuncu.sinif != "Büyücü":
                yavas_yaz("Büyü kullanmak için büyücü olmalısın.")
            else:
                basarili = oyuncu.buyu_kullan(dusman)
                if not basarili:
                    continue
        elif secim == "3":
            oyuncu.iyilestir()
        elif secim == "4":
            if random.random() < 0.5:
                yavas_yaz("Kaçmayı başardın!")
                return False
            else:
                yavas_yaz("Kaçamadın!")
        else:
            yavas_yaz("Geçersiz seçim.")
            continue

        if dusman.can <= 0:
            yavas_yaz(f"{dusman.isim} yenildi!")
            return True

        # Düşmanın saldırısı
        dusman.saldir(oyuncu)
        if oyuncu.can <= 0:
            yavas_yaz("Sen öldün!")
            return False

# === Ana Menü ===
def ana_menu():
    while True:
        yavas_yaz("\n--- Zindan Kaçışı ---")
        print("1. Yeni Oyun Başlat")
        print("2. Oyunu Yükle")
        print("3. Çıkış")
        secim = input("Seçimin: ")
        if secim == "1":
            isim = input("Karakter ismi gir: ")
            print("Sınıflar: Savaşçı, Okçu, Büyücü")
            sinif = input("Sınıf seç (varsayılan: Savaşçı): ")
            if sinif not in ["Savaşçı", "Okçu", "Büyücü"]:
                sinif = "Savaşçı"
            oyuncu = Karakter(isim, sinif)
            return oyuncu
        elif secim == "2":
            isim = input("Yüklenecek karakter ismi: ")
            oyuncu = yukle(isim)
            if oyuncu:
                return oyuncu
        elif secim == "3":
            yavas_yaz("Güle güle!")
            sys.exit()
        else:
            yavas_yaz("Geçersiz seçim.")

# === Oyun Başlat ===
def oyun_baslat():
    oyuncu = ana_menu()
    while True:
        oyuncu.durum()
        rastgele_olay(oyuncu)
        if oyuncu.can <= 0:
            yavas_yaz("Oyuncu öldü, oyun bitti.")
            break
        kaydet(oyuncu)

if __name__ == "__main__":
    oyun_baslat()
